In [9]:
#import the necessary libraries
import pandas as pd   
from matplotlib import pyplot
import numpy as np
import time


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier

In [10]:
df = pd.read_csv('train.csv')#reading the csv file train.csv

In [11]:
df = pd.get_dummies(df)
df.dtypes

RecordID                                     int64
hospital_id                                  int64
icu_id                                       int64
age                                        float64
elective_surgery                             int64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 96, dtype: object

In [12]:
X= df.loc[:, df.columns != 'hospital_death']
X

,RecordID,hospital_id,icu_id,age,elective_surgery,pre_icu_los_days,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,gcs_eyes_apache,...,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,1,126,1931,28.0,0,0.009722,122.0,703.03,0,3.0,...,0,0,0,1,0,0,0,0,0,0
1,2,112,1544,69.0,0,0.038194,302.0,109.09,0,4.0,...,1,0,0,0,0,0,0,0,0,0
2,3,153,1517,84.0,0,11.171528,106.0,212.01,0,3.0,...,0,0,0,0,0,0,1,0,0,0
3,4,109,1811,59.0,0,0.090972,301.0,403.01,0,4.0,...,0,0,0,0,1,0,0,0,0,0
4,5,287,1845,85.0,0,0.031944,117.0,106.01,0,4.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,110,1689,42.0,0,0.000000,308.0,1102.01,0,3.0,...,0,0,0,0,0,0,0,0,1,0
49996,49997,155,1719,61.0,0,4.824306,120.0,407.01,0,4.0,...,0,0,0,0,1,0,0,0,0,0
49997,49998,93,1912,50.0,0,0.000694,113.0,501.05,0,4.0,...,1,0,0,0,0,0,0,0,0,0
49998,49999,230,1720,54.0,1,0.420139,202.0,1211.01,1,4.0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
y = df[['hospital_death']]

In [14]:
X.fillna(X.median(), inplace=True)
imr = KNNImputer(n_neighbors=500, weights='distance')
imr = imr.fit(X.values)
imputed_data = imr.transform(X.values)
imputed_data

C:\Users\Zuhair Shah\AppData\Local\Temp\ipykernel_4076\329407764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)


array([[1.0000e+00, 1.2600e+02, 1.9310e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0000e+00, 1.1200e+02, 1.5440e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.0000e+00, 1.5300e+02, 1.5170e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [4.9998e+04, 9.3000e+01, 1.9120e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [4.9999e+04, 2.3000e+02, 1.7200e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [5.0000e+04, 2.8300e+02, 1.6830e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [15]:
num_of_models = 400
depth_level = 3

In [16]:
# knn_model = Pipeline([("scaler", RobustScaler()), ("knr", KNeighborsClassifier(n_neighbors=1075, weights='distance',p=1))])

# bg_c = BaggingClassifier(base_estimator=knn_model, n_estimators=num_of_models, n_jobs=-1)
# bg_c.fit(X,y)
# rf = RandomForestClassifier(max_depth=11,n_estimators=num_of_models, min_samples_leaf=10, max_leaf_nodes=1000)
gb = GradientBoostingClassifier(max_depth=3,n_estimators=400,  min_samples_leaf=10, max_leaf_nodes=1000)
ab = AdaBoostClassifier(n_estimators=15, learning_rate=0.5, base_estimator=gb)
ab.fit(X,y)

c:\Users\Zuhair Shah\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Zuhair Shah\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv('test.csv')
df1 = pd.get_dummies(df1)
df1.dtypes

RecordID                                     int64
hospital_id                                  int64
icu_id                                       int64
age                                        float64
elective_surgery                             int64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 95, dtype: object

In [ ]:
df1.fillna(df1.median(), inplace=True)
x_test=df1


In [ ]:
prob = ab.predict_proba(x_test)

In [ ]:
import csv
prob = prob[:,1]
filepath = 'output.csv'
with open(filepath, mode='w', newline='') as file: 
    writer = csv.writer(file)
    c=1
    for i in prob:
        writer.writerow([50000+c , i])
        c=c+1